In [1]:
import pandas as pd
from datasets import load_dataset
import json
from sentence_transformers import SentenceTransformer, util
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, TaskType, PeftModel
from trl import SFTTrainer
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig
import torch
from datasets import Dataset
import re
from rouge_score import rouge_scorer
from peft import PeftConfig, PeftModel
import torch_tensorrt
import modelopt
from peft import PeftModel

In [ ]:
# model_id = "./nairs-2e"
# quantization_config = BitsAndBytesConfig(load_in_4bit=True,
#                                          bnb_4bit_compute_dtype=torch.float16)
# use_quantization_config = True 
# if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
#   attn_implementation = "flash_attention_2"
# else:
#   attn_implementation = "sdpa"
# print(f"[INFO] Using attention implementation: {attn_implementation}")
# print(f"[INFO] Using model_id: {model_id}")
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
# llama = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id, 
#                                              torch_dtype = torch.float16,
#                                               quantization_config=quantization_config if use_quantization_config else None,
#                                                low_cpu_mem_usage=True,
#                                                  device_map = "auto",
#                                                 attn_implementation=attn_implementation,
#                                              return_dict = True,
#                                              torchscript = True,
#                                            )
# if not use_quantization_config:
#     llama.to("cuda")

In [2]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
model_id = "./nairs-2d"
peft_config = PeftConfig.from_pretrained(model_id)

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="cpu"  # load on CPU for now
)
model = PeftModel.from_pretrained(base_model, model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = model.merge_and_unload()

In [4]:
model.save_pretrained("./nairs_merged")

In [ ]:
model_id = "./nairs-merged"
model = AutoModelForCausalLM.from_pretrained("./nairs_merged",
                                 device_map = "cuda"
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/6 [00:15<?, ?it/s]

In [2]:
model_id = "./nairs_merged"
#base_model = "meta-llama/Llama-2-7b-chat-hf"
use_quantization_config = True 
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")
print(f"[INFO] Using model_id: {model_id}")
peft = PeftConfig.from_pretrained(model_id)
base_model = AutoModelForCausalLM.from_pretrained(peft.base_model_name_or_path,
                                                attn_implementation=attn_implementation,
                                                quantization_config=quantization_config if use_quantization_config else None,
                                                device_map = "auto",

                                                 )
model = PeftModel.from_pretrained(base_model,
                                  model_id,
                                 )
tokenizer = AutoTokenizer.from_pretrained(peft.base_model_name_or_path)
if not use_quantization_config:
    model.to("cuda")

[INFO] Using attention implementation: flash_attention_2
[INFO] Using model_id: ./nairs_merged


ValueError: Can't find 'adapter_config.json' at './nairs_merged'

In [4]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linea

In [6]:
# 2. Create strict wrapper
class ScriptableWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, input_ids):
        return self.model(
            input_ids=input_ids,
            attention_mask=None,
            position_ids=None,
            past_key_values=None,
            inputs_embeds=None,
            labels=None,
            use_cache=True,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            torchscript=None
        )[0]  # Return just logits

In [7]:
example_input = tokenizer("What is the principle of Relativity?", return_tensors="pt")["input_ids"].to("cuda")

# 4. Script the model
wrapped_model = ScriptableWrapper(model).eval()
with torch.no_grad():
    scripted_model = torch.jit.trace(wrapped_model, example_input)

In [10]:
compile_settings = {
    "inputs": [torch_tensorrt.Input(
        min_shape=[1, 1],    # Minimum input shape
        opt_shape=[1, 128],  # Optimal/typical shape
        max_shape=[1, 256], # Maximum shape
        dtype=torch.float16
    )],
    "enabled_precisions": {torch.float16},
    "workspace_size": 1 << 28,  # 256MB workspace instead of default 1GB
    "ir": "torchscript",
    "truncate_long_and_double": True
}

In [ ]:
nairs = torch_tensorrt.compile(scripted_model, **compile_settings)

In [8]:
example_input = tokenizer("What is the principle of Relativity?", return_tensors="pt")["input_ids"].to("cuda")
traced_model = torch.jit.trace(scripted_model, example_input)

  warnings.warn(



In [ ]:
def safe_compile(model, compile_settings, max_retries=3):
    for attempt in range(max_retries):
            
            # Gradually reduce workspace size on failures
            current_settings = compile_settings.copy()
            current_settings["workspace_size"] = 1 << (30 - attempt * 2)  # 1GB → 256MB → 64MB
            
            print(f"Attempt {attempt + 1} with workspace {current_settings['workspace_size'] >> 20}MB")
            return torch_tensorrt.compile(model, **current_settings)
    
# Usage:
nairs = safe_compile(
    scripted_model,
    compile_settings={
        "inputs": [torch_tensorrt.Input(
            min_shape=[1, 1],
            opt_shape=[1, 128],
            max_shape=[1, 256],
            dtype=torch.float16
        )],
        "enabled_precisions": {torch.float16},
        "truncate_long_and_double": True
    }
)

In [ ]:
compile_settings = {
    "inputs": [
        torch_tensorrt.Input(
            shape = [1,7],
            dtype = torch.float16,
            
        )
    ],
    "enabled_precisions": {torch.half},
    "ir": "torchscript"
}
nairs = torch_tensorrt.compile(scripted_model, **compile_settings)

In [ ]:
wrapped = ScriptableWrapper(model)
example_input = tokenizer("What is the principle of Relativity?", return_tensors="pt")["input_ids"].to("cuda")

In [ ]:
traced_model = torch.jit.trace(model, example_input)

In [ ]:
class TracedModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, input_ids):
        outputs = self.model(input_ids=input_ids)
        return outputs.logits  # Return only logits

wrapped_model = TracedModelWrapper(model)
traced_model = torch.jit.trace(wrapped_model, example_input)

In [ ]:
traced_model = torch.jit.script(wrapped_model)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# 1. Load original model
model_id = "./nairs-2d"
peft = PeftConfig.from_pretrained(model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    peft.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, model_id).eval()